In [22]:
import numpy as np

import glob
import re
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats.stats as stats

#from sklearn.decomposition import PCA

In [23]:
import zipfile

# read the dataset using the compression zip
df = pd.read_csv('all_data.zip',compression='zip')

In [24]:
df

,Comment,Author,PR_number,Date,Commit_message,Description,status
0,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,NaN,ABANDONED
1,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,NaN,MERGED
2,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,Bug 366528 - [Compatibility] ImplementIMenuSer...,ABANDONED
3,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,Bug 366528 - [Compatibility] ImplementIMenuSer...,MERGED
4,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,IMenuService#populateContributionManager(*)Wor...,NaN,ABANDONED
...,...,...,...,...,...,...,...
3943426,NaN,NaN,9585_rev1_,NaN,NaN,add jenkins rebalance tests,MERGED
3943427,NaN,NaN,9626_rev1_,NaN,NaN,adding timeout as an option to direct and prox...,MERGED
3943428,NaN,NaN,9670_rev1_,NaN,Added four upgradetests:-1- in-place upgrade f...,Add support for Multiple version upgradeAdded ...,MERGED
3943429,NaN,NaN,9705_rev1_,NaN,INI resource file can have entries as followin...,Add support for cluster tag in resource fileIN...,MERGED


In [25]:
df['status'] = np.where((df['status'] == 'MERGED') ,True,False)*1

In [26]:
df

,Comment,Author,PR_number,Date,Commit_message,Description,status
0,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,NaN,0
1,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,NaN,1
2,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,Bug 366528 - [Compatibility] ImplementIMenuSer...,0
3,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,Checked buckets for power of 2 (according to s...,Bug 366528 - [Compatibility] ImplementIMenuSer...,1
4,Patch Set 1: I would prefer that you didn't su...,Kenny Maynard,12027_rev1,2012-01-03 22:37:04.706000000,IMenuService#populateContributionManager(*)Wor...,NaN,0
...,...,...,...,...,...,...,...
3943426,NaN,NaN,9585_rev1_,NaN,NaN,add jenkins rebalance tests,1
3943427,NaN,NaN,9626_rev1_,NaN,NaN,adding timeout as an option to direct and prox...,1
3943428,NaN,NaN,9670_rev1_,NaN,Added four upgradetests:-1- in-place upgrade f...,Add support for Multiple version upgradeAdded ...,1
3943429,NaN,NaN,9705_rev1_,NaN,INI resource file can have entries as followin...,Add support for cluster tag in resource fileIN...,1


In [27]:
df['PR_number'].isna().sum()

0

In [34]:
text = ""

The Flesch-Kincaid Grade Level

In [32]:
import textstat

for row in df['Comment']:
    df['Flesch-Kincaid'] = textstat.flesch_reading_ease(row)
#textstat.flesch_reading_ease(text)

TypeError: expected string or bytes-like object

The Fog Scale (Gunning FOG Formula)

In [35]:
textstat.gunning_fog(text)

0.0

Automated Readability Index

In [ ]:
textstat.automated_readability_index(text)

12.2